In [1]:
import os
from dotenv import find_dotenv, load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [2]:
import boto3
client = boto3.client('textract',region_name='us-east-1',aws_access_key_id='----',aws_secret_access_key='----')

In [61]:
from PIL import Image, ImageDraw
from fpdf import FPDF
import os, json
from typing import List
from pdf2image import convert_from_path

# Store Pdf with convert_from_path function
images = convert_from_path('Original SOF Samples/27.pdf')
 
for i in range(len(images)):
      # Save pages as images in the pdf
    images[i].save('pdf2img/pdf27/pdf27-page'+ str(i) +'.jpg', 'JPEG')

for i, img in enumerate(os.listdir("pdf2img/pdf27")):
    
    with open(f'pdf2img/pdf27/{img}', 'rb') as image:
        bytes_img = bytearray(image.read())

    response = client.detect_document_text(
        Document={'Bytes': bytes_img}
    )

    text = ""
    for j, item in enumerate(response["Blocks"]):
        if item["BlockType"] == "LINE":
            print (item["Text"].lower())
            text = text + "\n"+item["Text"]
    
    if "\\" in text:
        text = text.replace("\\", "")

    with open(f"img2text/pdf27-tex/page{i}.txt", "w") as text_file:
        text_file.write(text)

pdf = FPDF()

for i, text in enumerate(os.listdir("img2text/pdf27-tex")):
    # Add a page
    pdf.add_page()
    
    # set style and size of font
    # that you want in the pdf
    pdf.set_font("Arial", size = 8)
    
    # open the text file in read mode
    f = open(f"img2text/pdf27-tex/{text}", "r", encoding="latin-1")
    
    # insert the texts in pdf
    for x in f:
        pdf.cell(200, 5, txt = x, ln = 1, align = 'C')

    f.close()
# save the pdf with name .pdf
pdf.output('new-pdfs/pdf27-tex.pdf', "F")

from langchain.chat_models import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser
from langchain_community.document_loaders import PyPDFLoader
from langchain.chains import create_extraction_chain, create_extraction_chain_pydantic

llm = ChatOpenAI()

_ = load_dotenv(find_dotenv())

class Document(BaseModel):

    vessel_name: str = Field(description="vessel_name")
    port_name: str = Field(description="port_name")
    terminal_berth: str = Field(description="terminal_berth")
    activity_type: str = Field(description="activity_type")
    cargo_commodity: str = Field(description="cargo commodity consignment")
    shipper_name_receiver_name: str = Field(description="shipper_name_receiver_name")
    B_L_quantity: str = Field(description="B/L_quantity")
    equipment_ship_shore_name: str = Field(description="equipment_ship_shore_name")
    stevedore_company: str = Field(description="stevedore_company")
    charterers_name: str = Field(description="charterers_name")
    agents_name: str = Field(description="agents name")
    next_port: str = Field(description="next_port")
    arr_draft_forward: str = Field(description="arr_draft_forward")
    arr_draft_after: str = Field(description="arr_draft_after")
    dep_draft_forward: str = Field(description="dep_draft_forward")
    dep_draft_after: str = Field(description="dep_draft_after")
    vessel_arrived_date: str = Field(description="vessel_arrived_date")
    vessel_arrived_time: str = Field(description="vessel_arrived_time")
    vessel_arrived_remarks: str = Field(description="vessel_arrived_remarks")
    NOR_tendered_date: str = Field(description="NOR_tendered_date")
    NOR_tendered_time: str = Field(description="NOR_tendered_time")
    NOR_tendered_remarks: str = Field(description="NOR_tendered_remarks")
    arrived_pilot_station_date: str = Field(description="arrived_pilot_station_date")
    arrived_pilot_station_time: str = Field(description="arrived_pilot_station_time")
    arrived_pilot_station_remarks: str = Field(description="arrived_pilot_station_remarks")
    POB_date: str = Field(description="POB_date pilot on board date")
    POB_time: str = Field(description="POB_time pilot on board time")
    POB_remarks: str = Field(description="POB_remarks pilot on board remarks")
    # pilot_on_board: str = Field(description="pilot on board")
    first_line_ashore_date: str = Field(description="first_line_ashore_date")
    first_line_ashore_time: str = Field(description="first_line_ashore_time")
    first_line_ashore_remarks: str = Field(description="first_line_ashore_remarks")
    all_fast_date: str = Field(description="all_fast_date")
    all_fast_time: str = Field(description="all_fast_time")
    all_fast_remarks: str = Field(description="all_fast_remarks")
    gangway_down_pilot_away_date: str = Field(description="gangway_down_pilot_away_date")
    gangway_down_pilot_away_time: str = Field(description="gangway_down_pilot_away_time")
    gangway_down_pilot_away_remakrs: str = Field(description="gangway_down_pilot_away_remarks")
    inward_clearance_started_date: str = Field(description="inward_clearance_started_date")
    inward_clearance_started_time: str = Field(description="inward_clearance_started_time")
    inward_clearance_started_remarks: str = Field(description="inward_clearance_started_remarks")
    inward_clearance_completed_date: str = Field(description="inward_clearance_completed_date")
    inward_clearance_completed_time: str = Field(description="inward_clearance_completed_time")
    inward_clearance_completed_remarks: str = Field(description="inward_clearance_completed_remarks")
    free_pratique_granted_date: str = Field(description="free_pratique_granted_date")
    free_pratique_granted_time: str = Field(description="free_pratique_granted_time")
    free_pratique_granted_remarks: str = Field(description="free_pratique_granted_remarks")
    initial_draft_survey_started_date: str = Field(description="initial_draft_survey_started_date")
    initial_draft_survey_started_time: str = Field(description="initial_draft_survey_started_time")
    initial_draft_survey_started_remarks: str = Field(description="initial_draft_survey_started_remarks")
    initial_draft_survey_completed_date: str = Field(description="initial_draft_survey_completed_date")
    initial_draft_survey_completed_time: str = Field(description="initial_draft_survey_completed_time")
    initial_draft_survey_completed_remarks: str = Field(description="initial_draft_survey_completed_remarks")
    commenced_loading_date: str = Field(description="commenced loading date")
    commenced_loading_time: str = Field(description="commenced loading time")
    commenced_loading_remarks: str = Field(description="commenced loading remarks")
    ceased_loading_date: str = Field(description="ceased date")
    ceased_loading_time: str = Field(description="ceased time")
    ceased_loading_remarks: str = Field(description="ceased remarks")
    all_clear_date: str = Field(description="all_clear_date")
    all_clear_time: str = Field(description="all_clear_time")
    all_clear_remarks: str = Field(description="all_clear_remarks")
    pilot_away_date: str = Field(description="pilot_away_date")
    pilot_away_time: str = Field(description="pilot_away_time")
    pilot_away_remarks: str = Field(description="pilot_away_remarks")
    resumed_loading_date: str = Field(description="resumed_loading_date")
    resumed_loading_time: str = Field(description="resumed_loading_time")
    resumed_loading_remarks: str = Field(description="resumed_loading_remarks")
    completed_loading_date: str = Field(description="completed loading date/ completed operation")
    completed_loading_time: str = Field(description="completed loading time/ completed operation")
    completed_loading_remarks: str = Field(description="completed loading remarks/ completed operation")
    final_draft_survey_commenced_date: str = Field(description="final_draft_survey_commenced_date")
    final_draft_survey_commenced_time: str = Field(description="final_draft_survey_commenced_time")
    final_draft_survey_commenced_remarks: str = Field(description="final_draft_survey_commenced_remarks")
    final_draft_survey_completed_date: str = Field(description="final_draft_survey_completed_date")
    final_draft_survey_completed_time: str = Field(description="final_draft_survey_completed_time")
    final_draft_survey_completed_remarks: str = Field(description="final_draft_survey_completed_remarks")
    cargo_documents_onboard_date: str = Field(description="cargo_documents_onboard_date")
    cargo_documents_onboard_time: str = Field(description="cargo_documents_onboard_time")
    cargo_documents_onboard_remarks: str = Field(description="cargo_documents_onboard_remarks")
    
def load_pdf():
    loader = PyPDFLoader("new-pdfs/pdf27-tex.pdf")
    pages = loader.load()
    return pages

def extract_metadata_from_pdf_output_parser():
    pages = load_pdf()
    
    parser = JsonOutputParser(pydantic_object=Document)
    
    prompt = PromptTemplate(
        template="Extract the information as specified.\n{format_instructions}\n{context}\n if no value exists for label, leave it empty\n Do not repeat same values for multiple labels",
        input_variables=["context"],
        partial_variables={"format_instructions": parser.get_format_instructions()},
    )
    
    chain = prompt | llm | parser
    
    response = chain.invoke({
        "context": pages
    })

    print("Function: extract_metadata_from_pdf_output_parser")
    with open('pdf2json/pdf27-tex-2.json', 'w', encoding='utf-8') as f:
        json.dump(response, f, ensure_ascii=False, indent=4)
    print(response)
    
def extract_metadata_from_pdf_extraction_chain():
    pages = load_pdf()
    
    schema = {
    "properties": {
        "vessel_name": {"type": "string"},
        "port_name": {"type": "string"},
        "terminal_berth": {"type": "string"},
        "activity_type": {"type": "string"},
        "cargo_commodity": {"type": "string"},
        "shipper_name_receiver_name": {"type": "string"},
        "B/L_quantity": {"type": "string"},
        "equipment_ship_shore_name": {"type": "string"},
        "stevedore_company": {"type": "string"},
        "charterers_name": {"type": "string"},
        "agents_name": {"type": "string"},
        "next_port": {"type": "string"},
        "arr_draft_forward": {"type": "string"},
        "arr_draft_after": {"type": "string"},
        "dep_draft_forward": {"type": "string"},
        "dep_draft_after": {"type": "string"},
        "vessel_arrived_date": {"type": "string"},
        "vessel_arrived_time": {"type": "string"},
        "vessel_arrived_remarks": {"type": "string"},
        "NOR_tendered_date": {"type": "string"},
        "NOR_tendered_time": {"type": "string"},
        "NOR_tendered_remarks": {"type": "string"},
        "arrived_pilot_station_date": {"type": "string"},
        "arrived_pilot_station_time": {"type": "string"},
        "arrived_pilot_station_remarks": {"type": "string"},
        "POB_date": {"type": "string"},
        "POB_time": {"type": "string"},
        "POB_remarks": {"type": "string"},
        # "pilot_on_board": {"type": "string"},
        "first_line_ashore_date": {"type": "string"},
        "first_line_ashore_time": {"type": "string"},
        "first_line_ashore_remarks": {"type": "string"},
        "all_fast_date": {"type": "string"},
        "all_fast_time": {"type": "string"},
        "all_fast_remarks": {"type": "string"},
        "gangway_down_pilot_away_date": {"type": "string"},
        "gangway_down_pilot_away_time": {"type": "string"},
        "gangway_down_pilot_away_remarks": {"type": "string"},
        "inward_clearance_started_date": {"type": "string"},
        "inward_clearance_started_time": {"type": "string"},
        "inward_clearance_started_remarks": {"type": "string"},
        "inward_clearance_completed_date": {"type": "string"},
        "inward_clearance_completed_time": {"type": "string"},
        "inward_clearance_completed_remarks": {"type": "string"},
        "free_pratique_granted_date": {"type": "string"},
        "free_pratique_granted_time": {"type": "string"},
        "free_pratique_granted_remarks": {"type": "string"},
        "initial_draft_survey_started_date": {"type": "string"},
        "initial_draft_survey_started_time": {"type": "string"},
        "initial_draft_survey_started_remarks": {"type": "string"},
        "initial_draft_survey_completed_date": {"type": "string"},
        "initial_draft_survey_completed_time": {"type": "string"},
        "initial_draft_survey_completed_remarks": {"type": "string"},
        "commenced_loading_date": {"type": "string"},
        "commenced_loading_time": {"type": "string"},
        "commenced_loading_remarks": {"type": "string"},
        "ceased_loading_date": {"type": "string"},
        "ceased_loading_time": {"type": "string"},
        "ceased_loading_remarks": {"type": "string"},
        "all_clear_date": {"type": "string"},
        "all_clear_time": {"type": "string"},
        "all_clear_remarks": {"type": "string"},
        "pilot_away_date": {"type": "string"},
        "pilot_away_time": {"type": "string"},
        "pilot_away_remarks": {"type": "string"},
        "resumed_loading_date": {"type": "string"},
        "resumed_loading_time": {"type": "string"},
        "resumed_loading_remarks": {"type": "string"},
        "completed_loading_date": {"type": "string"},
        "completed_loading_time": {"type": "string"},
        "completed_loading_remarks": {"type": "string"},
        "final_draft_survey_commenced_date": {"type": "string"},
        "final_draft_survey_commenced_time": {"type": "string"},
        "final_draft_survey_commenced_remarks": {"type": "string"},
        "final_draft_survey_completed_date": {"type": "string"},
        "final_draft_survey_completed_time": {"type": "string"},
        "final_draft_survey_completed_remarks": {"type": "string"},
        "cargo_documents_onboard_date": {"type": "string"},
        "cargo_documents_onboard_time": {"type": "string"},
        "cargo_documents_onboard_remarks": {"type": "string"},
    },

    "required": ["vessel voyage name", "port name", "terminal berth name", "activity type", "cargo/commodity/consignment", "shipper receiver name", "B/L quantity", "equipment ship shore name", \
                 "stevedore company", "charterers name", "agents name / agent", "next port", "Arr draft forward", "Arr draft after", "dep draft fwd", "dep draft aft", "vessel arrived date" \
                 "vessel arrived time", "vessel arrived remarks", "NOR tendered date", "NOR tendered time", "NOR tendered remarks", "arrived pilot station date", "arrived pilot station time", 
                 "arrived pilot station remarks", "pilot on board date", "pilot on board time", "pilot on board", "first line ashore date/commenced mooring", "first line ashore time/commenced mooring", \
                 "first line ashore remarks/commenced mooring", "all fast date", "all fast time", "all fast remarks", "Gangway Down and Pilot Away Date", "Gangway Down and Pilot Away time", \
                 "Gangway Down and Pilot Away remarks", "inward clearance started date", "inward clearance Started time", "inward clearance started remarks", "inward clearance completed date", \
                 "Inward Clearance completed date", "inward clearance completed time", "inward clearance completed remarks", "free pratique granted date", "free pratique granted time", \
                 "free pratique granted remarks", "initial draft survey finished/started date", "initial draft survey finished/started time", "initial draft survey finished/started remarks", \
                 "Initial Draft Survey completed date", "Initial Draft Survey completed time", "Initial Draft Survey completed remarks", \
                 "Commenced/completed loading discharging operation Date", "Commenced/completed loading discharging operation time", \
                 "Commenced/completed loading discharging operation remarks", "Ceased date", "Ceased time", "Ceased remarks", "all clear date", "all clear time", "all clear remarks", \
                 "pilot away date", "pilot away time", "pilot away remarks", "resumed loading/loading operation resumed date", \
                 "resumed loading/loading operation resumed time", "resumed loading/loading operation resumed remarks", "completed loading/discharging date", "completed loading/discharging time", \
                 "completed/pumping loading discharging remarks", "final draft survey commenced date", "final draft survey commenced time", "final draft survey commenced remarks", "final draft survey completed date", \
                 "final draft survey completed time", "final draft survey completed remarks", "cargo documents onboard date/completed documentation", \
                 "cargo documents onboard time/completed documentation", "cargo documents onboard remarks/completed documentation"]
    }
    
    chain = create_extraction_chain(schema, llm)

    response = chain.run(pages)
    
    print(response)
    
def extract_metadata_from_pdf_extraction_chain_pydantic():
    pages = load_pdf()

    chain = create_extraction_chain_pydantic(Document, llm)

    response = chain.run(pages)
    
    print("before")
    with open('data.json', 'w', encoding='utf-8') as f:
        json.dump(response, f, ensure_ascii=False, indent=4)
        print("here")
    print(response)
    
extract_metadata_from_pdf_output_parser()

statement of fact
austral asia line
agent: agenzia marittima alberto la rosa srl
port:
civitavecchia
vessel: aal newcastle
voyage:
berth: berth 23 terminal traiana
anchorage:
arrival date:
18th/11/2023
departure date:
19th/11/2023
date
time
eosp
18th/11/2023
1536
hrs
arrived pilot station
18th/11/2023
1536
hrs
pilot on board
18th/11/2023
1554
hrs
nor tendered
18th/11/2023
1536
hrs
first line
18th/11/2023
1647
hrs
all fast
18th/11/2023
1700
hrs
immigration clearance
18th/11/2023
1600
hrs
customs clearance
18th/11/2023
1600
hrs
commence operation
19th/11/2023
0800
hrs
complete operation
19th/11/2023
0915
hrs
pilot on board
19th/11/2023
hrs
last line/vessel sailed
19th/11/2023
1225
hrs
eta next port:
barl
hrs
tugs used
in 2
out 2
arrival draft
fwd
9,70
aft
10.10
departure draft
fwd
9,7
aft
191
bunkers rob eosp
hsfo
mgo
lsmgo
nil
bunkers on arrival (all fast)
hsfo
1.145,10
mgo
180,10
lsmgo
nil
bunker supplied
hsfo
nil
mgo
nil
lsmgo
nil
bunker on departure (stand by engine)
hsfo
2455.10
mgo